In [57]:
from numba import jit
import numpy as np

# Test numba and optimization

In [40]:
import numba
import platform
print('numba', numba.__version__)
print('processor', platform.processor())
print('python', platform.python_version(), platform.python_implementation())
print(platform.platform())

numba 0.44.1
processor x86_64
python 3.6.8 CPython
Linux-4.15.0-64-generic-x86_64-with-Ubuntu-18.04-bionic


In [42]:
import subprocess
# https://stackoverflow.com/a/47203612/8069403
print((subprocess.check_output("lscpu", shell=True).strip()).decode())

Architecture :                          x86_64
Mode(s) opératoire(s) des processeurs : 32-bit, 64-bit
Boutisme :                              Little Endian
Processeur(s) :                         8
Liste de processeur(s) en ligne :       0-7
Thread(s) par cœur :                    2
Cœur(s) par socket :                    4
Socket(s) :                             1
Nœud(s) NUMA :                          1
Identifiant constructeur :              GenuineIntel
Famille de processeur :                 6
Modèle :                                94
Nom de modèle :                         Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz
Révision :                              3
Vitesse du processeur en MHz :          2714.338
Vitesse maximale du processeur en MHz : 4000,0000
Vitesse minimale du processeur en MHz : 800,0000
BogoMIPS :                              6816.00
Virtualisation :                        VT-x
Cache L1d :                             32K
Cache L1i :                             32K
C

## Jit or no jit

In [43]:
b = .32

In [44]:
@jit(numba.float64[:](numba.float64[:]), nopython=True)
def thomas_attractor_jit(xyz):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0] += sin_xyz[1]
    dYdt[1] += sin_xyz[2]
    dYdt[2] += sin_xyz[0]
    return dYdt

xyz = np.random.rand(3)
_ = thomas_attractor_jit(xyz)

In [45]:
def thomas_attractor_nojit(xyz):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0] += sin_xyz[1]
    dYdt[1] += sin_xyz[2]
    dYdt[2] += sin_xyz[0]
    return dYdt

In [46]:
xyz = np.random.rand(3)

In [47]:
%%timeit
_ = thomas_attractor_nojit(xyz)

1.94 µs ± 68.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [48]:
%%timeit
_ = thomas_attractor_jit(xyz)

474 ns ± 9.45 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


* On Laptop:  

        5.17 µs ± 209 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)  with JIT
        32.6 µs ± 2.7 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)

        12.6 µs ± 5.67 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)  without copy
        4.68 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each) without copy and jit

        without optional argument b:
        3.85 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


* on work station: 

        no jit: 1.95 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
        Jit: 485 ns ± 6.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)  

### with b as argument

In [52]:
@jit(numba.float64[:](numba.float64[:], numba.float64), nopython=True)
def thomas_attractor_jit_b_arg(xyz, b):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0] += sin_xyz[1]
    dYdt[1] += sin_xyz[2]
    dYdt[2] += sin_xyz[0]
    return dYdt

xyz = np.random.rand(3)
_ = thomas_attractor_jit_b_arg(xyz, 0.21)

In [56]:
%%timeit
_ = thomas_attractor_jit_b_arg(xyz, 0.198)

# 516 ns ± 6.17 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

517 ns ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## With 2d arrays

[solve_ivp doc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html): "The vectorized implementation allows a faster approximation of the Jacobian by finite differences (required for stiff solvers)." so no use here

In [71]:
@jit(numba.float64[:, :](numba.float64[:, :], numba.float64), nopython=True)
def thomas_attractor_2d_arrays(xyz, b):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0, :] += sin_xyz[1, :]
    dYdt[1, :] += sin_xyz[2, :]
    dYdt[2, :] += sin_xyz[0, :]
    return dYdt

xyz = np.random.rand(3, 1)
_ = thomas_attractor_2d_arrays(xyz, 0.21)

In [73]:
%%timeit
_ = thomas_attractor_2d_arrays(xyz, 0.198)
# 708 ns ± 17.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

698 ns ± 7.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [74]:
xyz = np.random.rand(3, 100)

In [75]:
%%timeit
_ = thomas_attractor_2d_arrays(xyz, 0.21)

3.98 µs ± 77.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Other test

In [22]:
b = .32
#@jit(nopython=True)
def thomas_attractor_inline(xyz):
    return np.sin(xyz)[(1, 2, 0), ] - b*xyz

In [23]:
b = .32
@jit(nopython=True)
def thomas_attractor_toArray(xyz):
    sin_xyz = np.sin(xyz)
    dYdt = np.array([sin_xyz[1], sin_xyz[2], sin_xyz[0]]) - b*xyz
    return dYdt

In [24]:
b = .32
@jit(nopython=True)
def thomas_attractor_tupleOut(xyz):
    sin_xyz = np.sin(xyz)
    return (sin_xyz[1] - b*xyz[0],
            sin_xyz[2] - b*xyz[1],
            sin_xyz[0] - b*xyz[2])

In [25]:
# test
xyz = np.random.rand(3)
_ = thomas_attractor_inline(xyz)
print(_)

_ = thomas_attractor_toArray(xyz)
print(_)

_ = thomas_attractor_tupleOut(xyz)
print(_)

[0.5700253  0.58158319 0.00823979]
[0.5700253  0.58158319 0.00823979]
(0.5700252979337868, 0.5815831881356401, 0.008239788176361396)


In [26]:
%%timeit
_ = thomas_attractor_inline(xyz)

2.32 µs ± 22.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [28]:
%%timeit
_ = thomas_attractor_toArray(xyz)

575 ns ± 7.78 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [29]:
%%timeit
_ = thomas_attractor_tupleOut(xyz)

331 ns ± 7.75 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


processor x86_64
python 3.6.8


In [169]:

@jit(nopython=True)
def normed_thomas_attractor(xyz):
    ''' ODE for Thomas attractor
        xyz: point positions, shape (dim, )
    '''
    
    sin_xyz = np.sin(xyz)
    dYdt = -b*xyz
    dYdt[0] += sin_xyz[1]
    dYdt[1] += sin_xyz[2]
    dYdt[2] += sin_xyz[0]
    return dYdt/np.sqrt(np.sum(dYdt**2))

In [171]:
%%timeit
_ = normed_thomas_attractor(xyz)

4.49 µs ± 363 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
